# Notebook to create the dataset with historical features

This notebook is used to create the dataset composed by current features, plus historical ones.

Input: 
- < selected> the best dataset according to the previous Grid Search i.e., 3-FeatureSelection
- <h0> how much history should be remembered 
Output: 

- < "dataset/%d.pkl"> all datasets composed current features + historical ones

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
import sys
import os

In [2]:
def createHistory(Presentdf,history):
    
    features = list(Presentdf.columns)
    features.remove("ExpID")
    features.remove("Label")
    
    
    
    Dataset = np.array(Presentdf[features])
    IDs = list(Presentdf["ExpID"])
    Labels = list(Presentdf["Label"])
    
    
    HistoryIDs     = []
    HistoryDataset = []
    HistoryLabels  = []
    
    for i in range(history,len(Dataset)):
        Row = []

        for j in range(i-history,i+1):
            Row += list(Dataset[j])

        HistoryIDs.append(IDs[i])
        HistoryDataset.append(Row)
        HistoryLabels.append(Labels[i])
    
    HistoryFeatures = []
    for i in range(history+1):
        for feature in features:
            HistoryFeatures.append(feature + " t%d"%i)
    
    
    return HistoryDataset, HistoryLabels, HistoryIDs, HistoryFeatures

In [3]:
selected = 1
Presentdf = pickle.load(open("../3-FeatureSelection/dataset/%d"%selected+".pkl","rb"))

if not os.path.exists('dataset'):
    os.makedirs('dataset')    
    
#Create Historical Dataset from current only up to 10 previous cycles
for h0 in [0,1,2,3,4,5,6,7,8,9,10]:
    Xnp, Ynp, Id, HistoryFeatures = createHistory(Presentdf,h0)

    df = pd.DataFrame.from_records(Xnp)
    df.columns = HistoryFeatures
    df.insert(loc=0, column='ExpID', value=Id)
    df['Label']=Ynp
    pickle.dump(df,open("dataset/%d.pkl"%h0,"wb")) 
